### CNN on CIFR Assignment:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [2]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [3]:
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [4]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import TerminateOnNaN
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
import tensorflow_addons as tfa
import datetime
import os

In [5]:
batch_size = 128
num_classes = 10
# epochs = 300
num_filter = 12
compression = 0.5
dropout_rate = 0

In [6]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170508288/170498071 [==============================] - 2s 0us/step


In [7]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [8]:
num_filter = 36
dropout_rate = 0
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [9]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 36)   972         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 36)  144         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 36)   0           ['batch_normalization[0][0]']

In [10]:
datagen = ImageDataGenerator(rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,featurewise_std_normalization=True,horizontal_flip=True,fill_mode='nearest',zoom_range=0.2,shear_range=0.2)
gen_flow = datagen.flow(X_train,y_train,batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [14]:
def decay_learning(epochs,l_rate):
  if epochs>1:
    if (epochs+1)%3==0:
      l_rate = l_rate-(l_rate*0.1)
      print('every 5th epoch learning rate is decreased by 5%')
  return l_rate

In [15]:
filepath = "/content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_loss',verbose=1, save_best_only=True, mode='min')
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1)
# Reducelr_onplateau = ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1)
term_nan = TerminateOnNaN()
log_dir = os.path.join("logs",'fits', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True)
lr_scheduled = LearningRateScheduler(decay_learning,verbose=1)

In [12]:
model.compile(optimizer=SGD(0.01, momentum = 0.7),loss="categorical_crossentropy",metrics='accuracy')

In [ ]:
model.fit_generator(gen_flow, epochs=30, validation_data=(X_test,y_test),callbacks=[checkpoint,term_nan,earlystop])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/30
391/391 [==============================] - ETA: 0s - loss: 1.6892 - accuracy: 0.3751
Epoch 1: val_loss improved from inf to 1.51292, saving model to /content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-01.hdf5
391/391 [==============================] - 247s 581ms/step - loss: 1.6892 - accuracy: 0.3751 - val_loss: 1.5129 - val_accuracy: 0.4460
Epoch 2/30
391/391 [==============================] - ETA: 0s - loss: 1.4138 - accuracy: 0.4868
Epoch 2: val_loss did not improve from 1.51292
391/391 [==============================] - 225s 575ms/step - loss: 1.4138 - accuracy: 0.4868 - val_loss: 1.6239 - val_accuracy: 0.4481
Epoch 3/30
391/391 [==============================] - ETA: 0s - loss: 1.2644 - accuracy: 0.5424
Epoch 3: val_loss improved from 1.51292 to 1.50090, saving model to /content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-03.hdf5
391/391 [==============================] - 220s 562ms/step - loss: 1.2644 - accuracy: 0.5424 - val_lo

In [ ]:
model_json = model.to_json()
with open("/content/drive/MyDrive/24-Assignment/model-.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
#restoring the last model
from keras.models import load_model
model = load_model('/content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-26.hdf5')
from keras.preprocessing.image import ImageDataGenerator
model.fit_generator(gen_flow, steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint,term_nan,earlystop])
# model.save_weights(os.path.join(path, '72epochs.h5'))

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


391/390 [==============================] - ETA: 0s - loss: 0.5257 - accuracy: 0.8167
Epoch 1: val_loss did not improve from 0.57993
390/390 [==============================] - 225s 566ms/step - loss: 0.5257 - accuracy: 0.8167 - val_loss: 0.8424 - val_accuracy: 0.7374
Epoch 2/30
391/390 [==============================] - ETA: 0s - loss: 0.5210 - accuracy: 0.8191
Epoch 2: val_loss did not improve from 0.57993
390/390 [==============================] - 225s 577ms/step - loss: 0.5210 - accuracy: 0.8191 - val_loss: 0.9106 - val_accuracy: 0.7276
Epoch 3/30
391/390 [==============================] - ETA: 0s - loss: 0.5083 - accuracy: 0.8235
Epoch 3: val_loss did not improve from 0.57993
390/390 [==============================] - 220s 562ms/step - loss: 0.5083 - accuracy: 0.8235 - val_loss: 0.6070 - val_accuracy: 0.8020
Epoch 4/30
391/390 [==============================] - ETA: 0s - loss: 0.5028 - accuracy: 0.8255
Epoch 4: val_loss did not improve from 0.57993
390/390 [=========================

In [ ]:
#restoring the last model
from keras.models import load_model
model = load_model('/content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-25.hdf5')
from keras.preprocessing.image import ImageDataGenerator
model.fit_generator(gen_flow, steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint,term_nan,earlystop])
# model.save_weights(os.path.join(path, '72epochs.h5'))

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


391/390 [==============================] - ETA: 0s - loss: 0.3749 - accuracy: 0.8683
Epoch 1: val_loss did not improve from 0.45637
390/390 [==============================] - 224s 564ms/step - loss: 0.3749 - accuracy: 0.8683 - val_loss: 0.5707 - val_accuracy: 0.8190
Epoch 2/30
391/390 [==============================] - ETA: 0s - loss: 0.3731 - accuracy: 0.8701
Epoch 2: val_loss did not improve from 0.45637
390/390 [==============================] - 219s 561ms/step - loss: 0.3731 - accuracy: 0.8701 - val_loss: 0.5169 - val_accuracy: 0.8308
Epoch 3/30
391/390 [==============================] - ETA: 0s - loss: 0.3693 - accuracy: 0.8723
Epoch 3: val_loss did not improve from 0.45637
390/390 [==============================] - 220s 564ms/step - loss: 0.3693 - accuracy: 0.8723 - val_loss: 0.4955 - val_accuracy: 0.8372
Epoch 4/30
391/390 [==============================] - ETA: 0s - loss: 0.3670 - accuracy: 0.8731
Epoch 4: val_loss did not improve from 0.45637
390/390 [=========================

In [ ]:
#restoring the last model
from keras.models import load_model
model = load_model('/content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-18.hdf5')
from keras.preprocessing.image import ImageDataGenerator
model.fit_generator(gen_flow, steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint,term_nan,earlystop])
# model.save_weights(os.path.join(path, '72epochs.h5'))

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


391/390 [==============================] - ETA: 0s - loss: 0.3185 - accuracy: 0.8885
Epoch 1: val_loss did not improve from 0.39633
390/390 [==============================] - 225s 567ms/step - loss: 0.3185 - accuracy: 0.8885 - val_loss: 0.5511 - val_accuracy: 0.8284
Epoch 2/30
391/390 [==============================] - ETA: 0s - loss: 0.3161 - accuracy: 0.8898
Epoch 2: val_loss did not improve from 0.39633
390/390 [==============================] - 220s 564ms/step - loss: 0.3161 - accuracy: 0.8898 - val_loss: 0.4796 - val_accuracy: 0.8521
Epoch 3/30
391/390 [==============================] - ETA: 0s - loss: 0.3117 - accuracy: 0.8910
Epoch 3: val_loss did not improve from 0.39633
390/390 [==============================] - 220s 564ms/step - loss: 0.3117 - accuracy: 0.8910 - val_loss: 0.4102 - val_accuracy: 0.8654
Epoch 4/30
391/390 [==============================] - ETA: 0s - loss: 0.3118 - accuracy: 0.8929
Epoch 4: val_loss did not improve from 0.39633
390/390 [=========================

In [ ]:
#restoring the last model
from keras.models import load_model
model = load_model('/content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-13.hdf5')
keras.backend.set_value(model.optimizer.momentum, 0.7)
keras.backend.set_value(model.optimizer.lr, 0.001)
from keras.preprocessing.image import ImageDataGenerator
model.fit_generator(gen_flow, steps_per_epoch = X_train.shape[0]/batch_size, epochs = 30, validation_data =(X_test, y_test), callbacks = [checkpoint,term_nan,earlystop])
# model.save_weights(os.path.join(path, '72epochs.h5'))

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


391/390 [==============================] - ETA: 0s - loss: 0.2495 - accuracy: 0.9122
Epoch 1: val_loss improved from 0.38042 to 0.33429, saving model to /content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-01.hdf5
390/390 [==============================] - 250s 555ms/step - loss: 0.2495 - accuracy: 0.9122 - val_loss: 0.3343 - val_accuracy: 0.8899
Epoch 2/30
391/390 [==============================] - ETA: 0s - loss: 0.2372 - accuracy: 0.9171
Epoch 2: val_loss improved from 0.33429 to 0.32902, saving model to /content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-02.hdf5
390/390 [==============================] - 216s 552ms/step - loss: 0.2372 - accuracy: 0.9171 - val_loss: 0.3290 - val_accuracy: 0.8919
Epoch 3/30
391/390 [==============================] - ETA: 0s - loss: 0.2333 - accuracy: 0.9195
Epoch 3: val_loss did not improve from 0.32902
390/390 [==============================] - 220s 564ms/step - loss: 0.2333 - accuracy: 0.9195 - val_loss: 0.3

In [13]:
#restoring the last model
from keras.models import load_model
model = load_model('/content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-07.hdf5')
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,featurewise_std_normalization=True,horizontal_flip=True,fill_mode='nearest',zoom_range=0.2,shear_range=0.2)
gen_flow = datagen.flow(X_train,y_train,batch_size=batch_size)
# keras.backend.set_value(model.optimizer.momentum, 0.99)
# keras.backend.set_value(model.optimizer.lr, 0.001)
model.fit_generator(gen_flow, steps_per_epoch = X_train.shape[0]/batch_size, epochs = 20, validation_data =(X_test, y_test), callbacks = [checkpoint,term_nan,earlystop])

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any trainin

Epoch 1/20
391/390 [==============================] - ETA: 0s - loss: 0.1749 - accuracy: 0.9395
Epoch 1: val_loss improved from inf to 0.31265, saving model to /content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-01.hdf5
390/390 [==============================] - 247s 589ms/step - loss: 0.1749 - accuracy: 0.9395 - val_loss: 0.3126 - val_accuracy: 0.8999
Epoch 2/20
391/390 [==============================] - ETA: 0s - loss: 0.1706 - accuracy: 0.9414
Epoch 2: val_loss improved from 0.31265 to 0.30632, saving model to /content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-02.hdf5
390/390 [==============================] - 229s 585ms/step - loss: 0.1706 - accuracy: 0.9414 - val_loss: 0.3063 - val_accuracy: 0.9032
Epoch 3/20
391/390 [==============================] - ETA: 0s - loss: 0.1691 - accuracy: 0.9415
Epoch 3: val_loss did not improve from 0.30632
390/390 [==============================] - 222s 567ms/step - loss: 0.1691 - accuracy: 0.9415 - val_lo

In [16]:
model = load_model('/content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-09.hdf5')
keras.backend.set_value(model.optimizer.lr, 0.0001)
keras.backend.set_value(model.optimizer.momentum, 0.5)
bacth_size = 256
datagen = ImageDataGenerator(rotation_range = 5, horizontal_flip = True, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 5)
gen_flow = datagen.flow(X_train,y_train,batch_size=batch_size)
model.fit_generator(gen_flow, steps_per_epoch = X_train.shape[0]/batch_size, epochs = 10, validation_data =(X_test, y_test), callbacks = [checkpoint,term_nan,lr_scheduled,earlystop])


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


391/390 [==============================] - ETA: 0s - loss: 0.1211 - accuracy: 0.9577
Epoch 1: val_loss improved from inf to 0.29507, saving model to /content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-01.hdf5
390/390 [==============================] - 228s 575ms/step - loss: 0.1211 - accuracy: 0.9577 - val_loss: 0.2951 - val_accuracy: 0.9040 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/10
391/390 [==============================] - ETA: 0s - loss: 0.1223 - accuracy: 0.9586
Epoch 2: val_loss improved from 0.29507 to 0.29452, saving model to /content/drive/MyDrive/24-Assignment/Saved_Models/Model-1/Model1-weights-02.hdf5
390/390 [==============================] - 223s 570ms/step - loss: 0.1223 - accuracy: 0.9586 - val_loss: 0.2945 - val_accuracy: 0.9048 - lr: 1.0000e-04
every 5th epoch learning rate is decreased by 5%

Epoch 3: LearningRateScheduler setting learning rate to 8.999999772640876e-05.
Epoch 3/10
3